In [88]:
import numpy as np
import pandas as pd

titanic = pd.read_csv('./data/titanic_train.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [89]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


- 정확도의 문제점 : 이진 분류의 경우 데이터의 구성에 따라서 모델의 성능을 왜곡할 수 있다.

In [106]:
from sklearn.base import BaseEstimator

class UserClassifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    
    # 성별이 1이면 0, 아니면 1로 예측
    def predict(self, X):
        pred = np.zeros( (X.shape[0], 1))
        for idx in range(X.shape[0]):
            if X['Sex'].iloc[idx] == 1:
                pred[idx] = 0
            else:
                pred[idx] = 1
        return pred

In [95]:
titanic_label = titanic['Survived']
#titanic_data  = titanic.drop(['Survived'], axis = 1)

In [62]:
# 불필요한 피처 제거(PassengerId, Name, Ticket)

titanic_data = titanic_data.drop(['PassengerId', 'Name', 'Ticket'], axis = 1)
titanic_data

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.2500,NaN,S
1,1,female,38.0,1,0,71.2833,C85,C
2,3,female,26.0,0,0,7.9250,NaN,S
3,1,female,35.0,1,0,53.1000,C123,S
4,3,male,35.0,0,0,8.0500,NaN,S
...,...,...,...,...,...,...,...,...
886,2,male,27.0,0,0,13.0000,NaN,S
887,1,female,19.0,0,0,30.0000,B42,S
888,3,female,NaN,1,2,23.4500,NaN,S
889,1,male,26.0,0,0,30.0000,C148,C


In [63]:
# Preprocessing
# - age: 평균으로 채우기
titanic_data['Age'] = titanic_data['Age'].fillna(titanic_data['Age'].mean())
titanic_data['Age']
# - cabin : N 으로 채우기
titanic_data['Cabin'] = titanic_data['Cabin'].fillna('N')
titanic_data['Cabin']
# - Embarked : N 으로 채우기
titanic_data['Embarked'] = titanic_data['Embarked'].fillna('N')
titanic_data['Embarked']

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

In [64]:
titanic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     891 non-null    object 
 7   Embarked  891 non-null    object 
dtypes: float64(2), int64(3), object(3)
memory usage: 55.8+ KB


In [81]:
# 레이블 인코딩 ( Cabin, Sex, Embarked )
# 여기다가 for 문만 추가해주면 강사님이 해주신거랑 똑같이 나옴.
from sklearn.preprocessing import LabelEncoder

titanic_data['Cabin'] = titanic['Cabin'].str[:1]
titanic_data['Cabin'].value_counts()
cabin = ['C', 'B', 'D', 'E', 'A', 'F', 'G', 'T', 'N']
encoder = LabelEncoder()
encoder.fit(cabin)
cabin_labels = encoder.transform(cabin)

print(cabin_labels)

sex = ['male', 'female']
encoder = LabelEncoder()
encoder.fit(sex)
sex_labels = encoder.transform(sex)

print(sex_labels)

embarked = ['S', 'C', 'Q', 'N']
encoder  = LabelEncoder()
encoder.fit(embarked)
embarked_labels = encoder.transform(embarked)

print(embarked_labels)


[2 1 3 4 0 5 6 8 7]
[1 0]
[3 0 2 1]


In [92]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코딩 Cabin , Sex , Embarked
def drop_features(frm):
    frm.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return frm


def pre_processing(frm):
    frm['Age'].fillna(frm['Age'].mean(), inplace=True)
    frm['Cabin'].fillna('N', inplace=True)
    frm['Embarked'].fillna('N', inplace=True)
    return frm


def label_encoder(frm):
    frm['Cabin'] = frm['Cabin'].str[:1]
    features = ['Cabin', 'Sex', 'Embarked']
    for feature in features:
        encoder = LabelEncoder()
        encoder.fit(frm[feature])
        frm[feature] = encoder.transform(frm[feature])
    return frm

def transform_features(frm):
    frm = drop_features(frm)
    frm = pre_processing(frm)
    frm = label_encoder(frm)
    return frm

In [93]:
titanic_data_feature = transform_features(titanic_data)

In [94]:
titanic_data_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    int32  
 2   Age       891 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Cabin     891 non-null    int32  
 7   Embarked  891 non-null    int32  
dtypes: float64(2), int32(3), int64(3)
memory usage: 45.4 KB


In [96]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(titanic_data_feature,
                                                    titanic_label,
                                                   test_size = 0.2,
                                                   random_state = 10)

In [97]:
X_train.shape

(712, 8)

In [98]:
X_test.shape

(179, 8)

In [100]:
y_train.shape

(712,)

In [101]:
y_test.shape

(179,)

In [103]:
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

In [107]:
user_model = UserClassifier()
user_model.fit(X_train, y_train)

y_pred = user_model.predict(X_test)

print('accuracy {} '.format( accuracy_score(y_test, y_pred) ))

accuracy 0.8212290502793296 


In [108]:
titanic_data_feature['Sex'].value_counts()

1    577
0    314
Name: Sex, dtype: int64

In [116]:
def metrics_evaluation(y_test, y_pred):
    print("정확도(Accuracy) : {}\n정밀도(Precision) : {}\n재현율(Recall) : {}\n조화평균(F1) : {}"
         .format(accuracy_score(y_test, y_pred),
                 precision_score(y_test, y_pred),
                 recall_score(y_test, y_pred),
                 f1_score(y_test, y_pred)))

In [117]:
metrics_evaluation(y_test, y_pred)

정확도(Accuracy) : 0.8212290502793296
정밀도(Precision) : 0.7678571428571429
재현율(Recall) : 0.6935483870967742
조화평균(F1) : 0.7288135593220338


In [119]:
from sklearn.linear_model import LogisticRegression

logist_model = LogisticRegression()
logist_model.fit(X_train, y_train)
y_pred = logist_model.predict(X_test)

metrics_evaluation(y_test, y_pred)

정확도(Accuracy) : 0.8268156424581006
정밀도(Precision) : 0.746031746031746
재현율(Recall) : 0.7580645161290323
조화평균(F1) : 0.752


C:\Users\1234\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [122]:
# 교차 검증
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import make_scorer

fold = KFold(n_splits=20, 
            random_state = 1, 
            shuffle = True)
scoring = {
    'accuracy' : make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall'   : make_scorer(recall_score),
    'f1_score' : make_scorer(f1_score),
}


result = cross_validate(logist_model, X_train, y_train, cv=fold, scoring = scoring)

C:\Users\1234\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\1234\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

C:\Users\1234\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\1234\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

In [123]:
result.keys()

dict_keys(['fit_time', 'score_time', 'test_accuracy', 'test_precision', 'test_recall', 'test_f1_score'])

In [124]:
print( result['test_accuracy'].mean() ) # 20개 세트의 평균
print( result['test_precision'].mean() ) # 20개 세트의 평균
print( result['test_recall'].mean() ) # 20개 세트의 평균
print( result['test_f1_score'].mean() ) # 20개 세트의 평균


0.7922619047619048
0.7583165774284195
0.6917543567543568
0.7163523680085564


In [125]:
# [실습] - load_breast_cancer()
# 유방암 관련 데이터 - 양성을 양성으로 예측한 비율이 높을수록 좋은 성능 모형이라 판단할 수 있다.

# model_selection
# pre_processing

In [132]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
cancer.keys()
cancer.target

# 분류학습기
# LogisticRegression
# RandomForestClassifier

# 성능평가

# 교차검증 후 성능평가

# GridSearchCV

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,